In [1]:
import glob
import pandas as pd

In [2]:
path = "A:/Sp23 - Predictive Analytics/Conagra Data Files/Data"

In [3]:
file_list = glob.glob(path + "/IRI_POS_Tablespreads_20*.xlsx")

In [4]:
excel_list =[]

In [5]:
for file in file_list:
    excel_list.append(pd.read_excel(file))

In [6]:
excel_merged = pd.concat(excel_list, ignore_index = True)

In [7]:
ts_merged = pd.DataFrame(excel_merged)

In [8]:
#ts_merged.head()

In [9]:
ts_merged['Time'] = ts_merged["Time"].str[-8:]

In [10]:
ts_merged["Time"] = pd.to_datetime(ts_merged["Time"], format = '%m-%d-%y')

In [11]:
#ts_merged.head()

In [12]:
ts_attribute = pd.read_excel(path + "/IRI_POS_Tablespreads_Product Attribute File.xlsx")

In [13]:
ts_joined = pd.merge(ts_merged,ts_attribute,on ="UPC 13 digit",  how='left')

In [14]:
#ts_joined.head()

In [15]:
ts_joined = ts_joined.reset_index(drop=True)

In [16]:
ts_joined = ts_joined.drop(columns = ["Geography_y", "Time_y"])

In [55]:
ts_joined.dtypes

Geography_x                                    object
Time_x                                 datetime64[ns]
Product Description                            object
UPC 13 digit                                    int64
Dollar Sales No Merch                         float64
Dollar Sales Any Merch                        float64
Unit Sales No Merch                           float64
Unit Sales Any Merch                          float64
Volume Sales No Merch                         float64
Volume Sales Any Merch                        float64
Price per Unit                                float64
Price per Unit No Merch                       float64
Price per Unit Any Merch                      float64
Price per Volume                              float64
Price per Volume No Merch                     float64
Price per Volume Any Merch                    float64
ACV Weighted Distribution No Merch            float64
ACV Weighted Distribution Any Merch           float64
Base Unit Sales             

In [18]:
ts_required = ts_joined[["Volume Sales No Merch", "CAG Manufacturer Value", "Price per Volume No Merch", "Geography_x"]]

In [19]:
ts_required.dtypes

Volume Sales No Merch        float64
CAG Manufacturer Value        object
Price per Volume No Merch    float64
Geography_x                   object
dtype: object

In [20]:
ts_conagra = ts_required.loc[(ts_required["CAG Manufacturer Value"] == "CONAGRA BRANDS")]

In [21]:
print(ts_conagra["CAG Manufacturer Value"].unique())

['CONAGRA BRANDS']


In [22]:
print(ts_required["Geography_x"].unique())

['Total US - Multi Outlet + Conv'
 'California - IRI Standard - Multi Outlet + Conv'
 'Great Lakes - IRI Standard - Multi Outlet + Conv'
 'Mid-South - IRI Standard - Multi Outlet + Conv'
 'Northeast - IRI Standard - Multi Outlet + Conv'
 'Plains - IRI Standard - Multi Outlet + Conv'
 'South Central - IRI Standard - Multi Outlet + Conv'
 'Southeast - IRI Standard - Multi Outlet + Conv'
 'West - IRI Standard - Multi Outlet + Conv']


In [23]:
ts_conagra

,Volume Sales No Merch,CAG Manufacturer Value,Price per Volume No Merch,Geography_x
0,1.241794e+06,CONAGRA BRANDS,1.011426,Total US - Multi Outlet + Conv
1,1.525283e+05,CONAGRA BRANDS,0.988020,Total US - Multi Outlet + Conv
2,1.736027e+04,CONAGRA BRANDS,1.002306,Total US - Multi Outlet + Conv
3,3.054180e+05,CONAGRA BRANDS,0.941813,Total US - Multi Outlet + Conv
4,2.139030e+05,CONAGRA BRANDS,1.683936,Total US - Multi Outlet + Conv
...,...,...,...,...
1316268,4.120045e+01,CONAGRA BRANDS,7.556513,West - IRI Standard - Multi Outlet + Conv
1316269,1.522821e+04,CONAGRA BRANDS,5.176604,West - IRI Standard - Multi Outlet + Conv
1316270,1.208547e+03,CONAGRA BRANDS,5.346315,West - IRI Standard - Multi Outlet + Conv
1316271,4.822417e+02,CONAGRA BRANDS,5.706442,West - IRI Standard - Multi Outlet + Conv


In [24]:
import numpy as np

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [26]:
# encode categorical variables using one-hot encoding
ts_conagra = pd.get_dummies(ts_conagra, columns=['Geography_x'],drop_first=True)

In [27]:
ts_conagra = ts_conagra.replace([np.inf, -np.inf], np.nan)
ts_conagra = ts_conagra.dropna()
ts_conagra = ts_conagra.reset_index()

In [28]:
ts_conagra.dtypes

index                                                               int64
Volume Sales No Merch                                             float64
CAG Manufacturer Value                                             object
Price per Volume No Merch                                         float64
Geography_x_Great Lakes - IRI Standard - Multi Outlet + Conv        uint8
Geography_x_Mid-South - IRI Standard - Multi Outlet + Conv          uint8
Geography_x_Northeast - IRI Standard - Multi Outlet + Conv          uint8
Geography_x_Plains - IRI Standard - Multi Outlet + Conv             uint8
Geography_x_South Central - IRI Standard - Multi Outlet + Conv      uint8
Geography_x_Southeast - IRI Standard - Multi Outlet + Conv          uint8
Geography_x_Total US - Multi Outlet + Conv                          uint8
Geography_x_West - IRI Standard - Multi Outlet + Conv               uint8
dtype: object

In [29]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ts_conagra.drop(['index','Volume Sales No Merch','CAG Manufacturer Value'], axis=1), ts_conagra['Volume Sales No Merch'], test_size=0.2, random_state=42)

In [30]:
# build the regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

LinearRegression()

In [31]:
# evaluate the model on the testing data
score = reg_model.score(X_test, y_test)
print('R-squared value:', score)

R-squared value: 0.0947542615582242


In [32]:
# provide a regression equation for predictions
intercept = reg_model.intercept_
coefficients = reg_model.coef_

In [33]:
print(intercept)
print(coefficients)

31671.18885126884
[-6892.09361255  3426.22805639  7482.9318352   3151.26130755
   818.43864286  3458.19398109  7699.82402937 48338.45943016
 -1020.05242531]


In [34]:
# merchendise effect

In [35]:
#ts_joined.dtypes

In [36]:
merch = ts_joined[["Volume Sales No Merch", "Volume Sales Any Merch", "Price per Volume No Merch", "Price per Volume Any Merch", "CAG Manufacturer Value", "CAG Form Value"]]

In [37]:
merch_conagra = merch.loc[(merch["CAG Manufacturer Value"] == "CONAGRA BRANDS")]

In [38]:
merch_conagra.head()

,Volume Sales No Merch,Volume Sales Any Merch,Price per Volume No Merch,Price per Volume Any Merch,CAG Manufacturer Value,CAG Form Value
0,1.241794e+06,182993.872299,1.011426,0.920339,CONAGRA BRANDS,ALL OTHER FORM
1,1.525283e+05,17263.338023,0.988020,0.902127,CONAGRA BRANDS,STICKS
2,1.736027e+04,717.499949,1.002306,0.898114,CONAGRA BRANDS,STICKS
3,3.054180e+05,16623.080285,0.941813,0.847538,CONAGRA BRANDS,STICKS
4,2.139030e+05,35242.316041,1.683936,1.493064,CONAGRA BRANDS,TUBS


In [39]:
merch_conagra["%_inc_in_sales"] = (merch_conagra["Volume Sales Any Merch"]/merch_conagra["Volume Sales No Merch"])*100
merch_conagra["reduced_price"] = merch_conagra["Price per Volume No Merch"] - merch_conagra["Price per Volume Any Merch"]

C:\Users\arshi\AppData\Local\Temp\ipykernel_15756\3318527985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merch_conagra["%_inc_in_sales"] = (merch_conagra["Volume Sales Any Merch"]/merch_conagra["Volume Sales No Merch"])*100
C:\Users\arshi\AppData\Local\Temp\ipykernel_15756\3318527985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merch_conagra["reduced_price"] = merch_conagra["Price per Volume No Merch"] - merch_conagra["Price per Volume Any Merch"]


In [40]:
merch_conagra.head()

,Volume Sales No Merch,Volume Sales Any Merch,Price per Volume No Merch,Price per Volume Any Merch,CAG Manufacturer Value,CAG Form Value,%_inc_in_sales,reduced_price
0,1.241794e+06,182993.872299,1.011426,0.920339,CONAGRA BRANDS,ALL OTHER FORM,14.736252,0.091086
1,1.525283e+05,17263.338023,0.988020,0.902127,CONAGRA BRANDS,STICKS,11.318121,0.085893
2,1.736027e+04,717.499949,1.002306,0.898114,CONAGRA BRANDS,STICKS,4.133000,0.104192
3,3.054180e+05,16623.080285,0.941813,0.847538,CONAGRA BRANDS,STICKS,5.442731,0.094275
4,2.139030e+05,35242.316041,1.683936,1.493064,CONAGRA BRANDS,TUBS,16.475843,0.190873


In [41]:
merch_conagra = merch_conagra.drop("CAG Manufacturer Value", axis=1)
merch_conagra.head()

,Volume Sales No Merch,Volume Sales Any Merch,Price per Volume No Merch,Price per Volume Any Merch,CAG Form Value,%_inc_in_sales,reduced_price
0,1.241794e+06,182993.872299,1.011426,0.920339,ALL OTHER FORM,14.736252,0.091086
1,1.525283e+05,17263.338023,0.988020,0.902127,STICKS,11.318121,0.085893
2,1.736027e+04,717.499949,1.002306,0.898114,STICKS,4.133000,0.104192
3,3.054180e+05,16623.080285,0.941813,0.847538,STICKS,5.442731,0.094275
4,2.139030e+05,35242.316041,1.683936,1.493064,TUBS,16.475843,0.190873


In [42]:
print(merch_conagra["CAG Form Value"].unique())

['ALL OTHER FORM' 'STICKS' 'TUBS' 'SPRAY/SQUEEZE']


In [43]:
# encode categorical variables using one-hot encoding
merch_conagra = pd.get_dummies(merch_conagra, columns=['CAG Form Value'],drop_first=True)

In [44]:
merch_conagra.dtypes

Volume Sales No Merch           float64
Volume Sales Any Merch          float64
Price per Volume No Merch       float64
Price per Volume Any Merch      float64
%_inc_in_sales                  float64
reduced_price                   float64
CAG Form Value_SPRAY/SQUEEZE      uint8
CAG Form Value_STICKS             uint8
CAG Form Value_TUBS               uint8
dtype: object

In [45]:
merch_conagra = merch_conagra.drop(["Volume Sales No Merch", "Volume Sales Any Merch", "Price per Volume No Merch", "Price per Volume Any Merch"], axis=1)

In [46]:
merch_conagra = merch_conagra.replace([np.inf, -np.inf], np.nan)
merch_conagra = merch_conagra.dropna()
merch_conagra = merch_conagra.reset_index()

In [47]:
merch_conagra.dtypes

index                             int64
%_inc_in_sales                  float64
reduced_price                   float64
CAG Form Value_SPRAY/SQUEEZE      uint8
CAG Form Value_STICKS             uint8
CAG Form Value_TUBS               uint8
dtype: object

In [48]:
corr = merch_conagra['%_inc_in_sales'].corr(merch_conagra['reduced_price'])

In [53]:
corr

0.025114953402102836

In [49]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(merch_conagra.drop(["index", "%_inc_in_sales"],axis =1), merch_conagra['%_inc_in_sales'], test_size=0.2, random_state=42)

In [50]:
reg_model_merch = LinearRegression()
reg_model_merch.fit(X_train, y_train)

LinearRegression()

In [54]:
# evaluate the model on the testing data
score_merch = reg_model_merch.score(X_test, y_test)

In [52]:
# provide a regression equation for predictions
intercept_merch = reg_model_merch.intercept_
coefficients_merch = reg_model_merch.coef_

print(intercept_merch)
print(coefficients_merch)

-11.390012000302335
[380.15317761 372.84851657 162.49948683 -12.62521698]
